In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clrs
import matplotlib.cm as cmx
import matplotlib.colorbar as cb
import emcee
import corner
import batman
import copy
import time
from astropy import units as u
from astropy.constants import G
from astropy.io import ascii
from matplotlib import gridspec
import scipy.optimize as sciop

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

%matplotlib inline

In [2]:
kep = ascii.read("./KOIarchive_stellar.csv")

In [3]:
print kep.colnames

['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_disp_prov', 'koi_comment', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_time0', 'koi_time0_err1', 'koi_time0_err2', 'koi_eccen', 'koi_eccen_err1', 'koi_eccen_err2', 'koi_longp', 'koi_longp_err1', 'koi_longp_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_ingress', 'koi_ingress_err1', 'koi_ingress_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_ror', 'koi_ror_err1', 'koi_ror_err2', 'koi_srho', 'koi_srho_err1', 'koi_srho_err2', 'koi_fittype', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_sma', 'koi_sma_err1', 'koi_sma_err2', 'koi_incl', 'koi_incl_err1', 'koi_incl_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol', 'koi_

In [4]:
kep.remove_columns(['koi_vet_stat','koi_vet_date','koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec','koi_disp_prov',
                    'koi_comment',
                    'koi_fwm_stat_sig','koi_fwm_sra','koi_fwm_sra_err','koi_fwm_sdec','koi_fwm_sdec_err','koi_fwm_srao','koi_fwm_srao_err',
                   'koi_fwm_sdeco','koi_fwm_sdeco_err','koi_fwm_prao','koi_fwm_prao_err','koi_fwm_pdeco','koi_fwm_pdeco_err',
                   'koi_dicco_mra','koi_dicco_mra_err','koi_dicco_mdec','koi_dicco_mdec_err','koi_dicco_msky','koi_dicco_msky_err',
                   'koi_dikco_mra','koi_dikco_mra_err','koi_dikco_mdec','koi_dikco_mdec_err','koi_dikco_msky','koi_dikco_msky_err'])

In [5]:
print kep

rowid  kepid   kepoi_name  kepler_name  ... dist_err1 dist_err2 nconfp nkoi
----- -------- ---------- ------------- ... --------- --------- ------ ----
    1 10797460  K00752.01  Kepler-227 b ...    131.26     83.53      2    2
    2 10797460  K00752.02  Kepler-227 c ...    131.26     83.53      2    2
    5 10854555  K00755.01  Kepler-664 b ...    514.88    205.95      1    1
    6 10872983  K00756.01  Kepler-228 d ...     516.6     184.5      3    3
    7 10872983  K00756.02  Kepler-228 c ...     516.6     184.5      3    3
    8 10872983  K00756.03  Kepler-228 b ...     516.6     184.5      3    3
   10 10910878  K00757.01  Kepler-229 c ...     54.25     88.15      3    3
   11 11446443  K00001.01    Kepler-1 b ...      9.76      9.76      1    1
   12 10666592  K00002.01    Kepler-2 b ...     16.66     21.21      1    1
   13  6922244  K00010.01    Kepler-8 b ...     77.54     77.54      1    1
  ...      ...        ...           ... ...       ...       ...    ...  ...
 9534 100281

In [30]:
x = "x"
print type(x)

for rowIdx in range(0,1):
    rowstr = ""
    for colIdx in range(0,len(kep.colnames)):
        print kep[kep.colnames[colIdx]][rowIdx]
        print type(kep[kep.colnames[colIdx]][rowIdx])
        if isinstance(kep[kep.colnames[colIdx]][rowIdx], str) or isinstance(kep[kep.colnames[colIdx]][rowIdx], np.ma.core.MaskedConstant):
            rowstr = rowstr+'"'+str(kep.colnames[colIdx])+'"'+':"{0}"'.format(kep[kep.colnames[colIdx]][rowIdx]) + ", "
        else:
            rowstr = rowstr+'"'+str(kep.colnames[colIdx])+'"'+':{0}'.format(kep[kep.colnames[colIdx]][rowIdx]) + ", "
    rowstr = rowstr[:-2]
    rowstr = "{" + rowstr + "},\n"
    print rowstr

<type 'str'>
1
<type 'numpy.int64'>
10797460
<type 'numpy.int64'>
K00752.01
<type 'numpy.string_'>
Kepler-227 b
<type 'numpy.string_'>
CONFIRMED
<type 'numpy.string_'>
Active
<type 'numpy.string_'>
2017-05-10
<type 'numpy.string_'>
CANDIDATE
<type 'numpy.string_'>
1.0
<type 'numpy.float64'>
0
<type 'numpy.int64'>
0
<type 'numpy.int64'>
0
<type 'numpy.int64'>
0
<type 'numpy.int64'>
q1_q17_dr25_koi
<type 'numpy.string_'>
NO_COMMENT
<type 'numpy.string_'>
9.48803557
<type 'numpy.float64'>
2.775e-05
<type 'numpy.float64'>
-2.775e-05
<type 'numpy.float64'>
170.53875
<type 'numpy.float64'>
0.00216
<type 'numpy.float64'>
-0.00216
<type 'numpy.float64'>
2455003.539
<type 'numpy.float64'>
0.00216
<type 'numpy.float64'>
-0.00216
<type 'numpy.float64'>
0
<type 'numpy.int64'>
--
<class 'numpy.ma.core.MaskedConstant'>
--
<class 'numpy.ma.core.MaskedConstant'>
--
<class 'numpy.ma.core.MaskedConstant'>
--
<class 'numpy.ma.core.MaskedConstant'>
--
<class 'numpy.ma.core.MaskedConstant'>
0.146
<type 'nu

In [7]:
with open("./KOIarchive_stellar.js","a") as outfile:
    outfile.write ("var planets = [\n")
    for rowIdx in range(0,len(kep)):
        rowstr = ""
        for colIdx in range(0,len(kep.colnames)):
            if isinstance(kep[kep.colnames[colIdx]][rowIdx], str) or isinstance(kep[kep.colnames[colIdx]][rowIdx], np.ma.core.MaskedConstant):
                rowstr = rowstr+'"'+str(kep.colnames[colIdx])+'"'+':"{0}"'.format(kep[kep.colnames[colIdx]][rowIdx]) + ", "
            else:
                rowstr = rowstr+'"'+str(kep.colnames[colIdx])+'"'+':{0}'.format(kep[kep.colnames[colIdx]][rowIdx]) + ", "
        rowstr = rowstr[:-2]
        rowstr = "{" + rowstr + "},\n"
        outfile.write(rowstr)
    outfile.write("];")
    outfile.close()